# nbed library: Binning and Center-of-Mass Example
L. Houben, Weizmann Institute of Science
last updated January 2026

- Load a 4D STEM nanobeam diffraction data set.
  This script uses a data set that can be downloaded from https://doi.org/10.5281/zenodo.15212905
- Inspect a frame.
- Bin the data for faster processing
- Calculate and display the center of mass

In [ ]:
import numpy as np
import numpy.matlib
#from objbrowser import browse
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pims
import trackpy as tp
# for animated view
import time
from IPython import display
%matplotlib inline
import os
# load nbed module and helper routines
# possibly installed under miniforge3/envs/py4Dstem/lib/python3.9/site-packages
import nbed
from nbed import ParabolaFit2D,bytscl

## Define filename, create class instance, load the data set, and display a single frame 
Currently supported formats: 
- PantaRhei .prz (default)
- EMPAD .raw
- sbin and qbin are the binning factors for the real and momentum space

In [ ]:
# Path definition and filename
# export format
path="/Users/houben/Downloads/"
filebasename="Coccolithophore0025"
filesuffix="prz"
sbin=2 # spatial binning
qbin=1 # momentum space binning

In [ ]:
# create an instance of the pyNBED module and load data
myset=nbed.pyNBED()
myset.LoadFile(path+filebasename+'.'+filesuffix)

### Bin the 4D dataset

In [ ]:
# spatial binning
if sbin > 1:
    myset.SpatialBinning(bin=sbin)

In [ ]:
# frame binning
if qbin > 1:
    myset.FrameBinning(bin=qbin)

### Create a virtual aperture image and display a single diffraction frame

In [ ]:
mask,adf=myset.VirtualApertureImage(radius=[0,10],invert=True)

In [ ]:
# plot the image
#dispimage=np.log(vimage)
dispimage=adf
plt.figure()
#subplot(r,c) provide the no. of rows and columns
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
# use the created array to output your multiple images. In this case I have stacked 4 images vertically
ax1.imshow(bytscl(dispimage,vmin=np.mean(dispimage)-3*np.std(dispimage),vmax=np.mean(dispimage)+3*np.std(dispimage)),cmap='gray',origin="lower") 
ax1.set_title('Virtual Image')
ax2.imshow(mask,cmap='gray',origin="lower")
ax2.set_title('Mask')
plt.show()

In [ ]:
# Display a single frame at row 174, column 94
frame=myset.ShowFrame(i=174//sbin, j=94//sbin,Log=True, sd=6)

### Create a centre-of-mass (COM) map 

In [ ]:
centre, comdata=myset.Com(mask=None, optimize=True)

In [ ]:
# The comdata is a complex array with the horizontal displacement in the real part
comx=np.real(comdata)
comy=np.imag(comdata)

In [ ]:
# Create a figure with the the x and y components of the COM
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Display first image
axes[0].imshow(comx, origin='lower')
axes[0].set_title("COM-x")

# Display second image
axes[1].imshow(comy,origin='lower')
axes[1].set_title("COM-y")

plt.tight_layout()
plt.show()


In [ ]:
# Use the internal function to map the COM data to a HSV color-coded image 
im=myset.Complex2HSVPlot(comdata, np.abs(comdata),0.0*np.max(np.abs(comdata)), 0.4*np.max(np.abs(comdata)),multiplicity=1,saturation=1.)